In [ ]:
import netCDF4
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Define your stations, corresponding node numbers, and custom names
stations_and_nodes = [
    {'station': '8632200', 'node': 104845, 'name': 'Kiptopeke, VA'}, #COORD: 37.1653, -75.9883
    {'station': '8571892', 'node': 12948, 'name': 'Cambridge, MD'}, #COORD: 38.5717, -76.0683
    {'station': '8516945', 'node': 52435, 'name': 'Kings Point, NY'}, #COORD: 40.8100, -73.7650
    {'station': '8722956', 'node': 76314, 'name': 'South Port, FL'}, #COORD: 26.0817, -80.1167
    {'station': '8720218', 'node': 90753, 'name': 'Mayport, FL'}, #COORD: 30.3917, -81.3983
    {'station': '8656483', 'node': 88800, 'name': 'Beaufort, NC'} #COORD: 34.7183, -76.6716
    # {'station': '8771341', 'node': 21927, 'name': 'GBay Entr., TX'}, #COORD: 29.3566, -94.7250
    # {'station': '8661070', 'node': 103187, 'name': 'Springmaid Pier, SC'}, #COORD: 33.655556, -78.916389
    # {'station': '8721604', 'node': 89105, 'name': 'Trident Pier, FL'}, #COORD: 28.415833, -80.593056
    # {'station': '8726724', 'node': 38938, 'name': 'Clearwater Beach, FL'}, #COORD: 27.978333, -82.831667
]

# Load netCDF files
withoutoffset = netCDF4.Dataset(r'/scratch/07174/soelem/without_atm/fort.63.nc', 'r')

# After loading the NetCDF file
zeta_shape = withoutoffset.variables['zeta'].shape
print(f"Shape of 'zeta' variable: {zeta_shape}")

for i, station_info in enumerate(stations_and_nodes):
    node_index = station_info['node']
    # Check if the node index is within the bounds for the 'zeta' variable
    if node_index < zeta_shape[1]:  # Assuming the node index is along the second dimension
        withoutoffset_data = withoutoffset.variables['zeta'][:, node_index]
        # Proceed with data handling and plotting as before
    else:
        print(f"Node index {node_index} for {station_info['name']} is out of bounds.")

# Setup the figure for subplots with adjusted figsize for wider and shorter appearance
fig, axs = plt.subplots(len(stations_and_nodes), 1, figsize=(26, 10), dpi=600, sharex=True)

for i, station_info in enumerate(stations_and_nodes):
    # Fetch NOAA tide data for each station
    url = f'https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?begin_date=20230716&end_date=20231031&station={station_info["station"]}&product=hourly_height&datum=MSL&time_zone=lst&units=metric&format=csv'
    noaa = pd.read_csv(url).dropna().replace(-99999, pd.NA).dropna()
    
    # Extract 'zeta' variable for the node
    withoutoffset_data = withoutoffset.variables['zeta'][:, station_info['node']]
    
    # Convert to DataFrame
    df_without_offset = pd.DataFrame(withoutoffset_data, columns=['Without Offset']).shift(-4)
    
    # Create date range for index
    date = pd.date_range(start='2023-07-16 00:00:00', periods=len(df_without_offset), freq='60min')
    df_without_offset['Date Time'] = date
    df_without_offset['Observation'] = noaa[' Water Level'].values[:len(df_without_offset)]
    
    # Plotting on subplot
    axs[i].plot(df_without_offset['Date Time'], df_without_offset['Observation'], linestyle='-', linewidth=5, color='black', label='NOAA Observation' if i == len(stations_and_nodes) - 1 else "")
    axs[i].plot(df_without_offset['Date Time'], df_without_offset['Without Offset'], linestyle='-', linewidth=5, color='magenta', label='ADCIRC Simulation' if i == len(stations_and_nodes) - 1 else "")
    
    axs[i].set_ylabel(f'{station_info["name"]}', fontsize=13)
    axs[i].xaxis.set_major_formatter(mdates.DateFormatter('%d-%b'))  # Set date format
    axs[i].tick_params(axis='x', labelrotation=45, labelsize=20)  # Increase x-tick font size
    axs[i].tick_params(axis='y', labelsize=15)  # Increase y-tick font size
    axs[i].grid(True)
    axs[i].set_facecolor('#E6E6FA')  # Light blue background for each subplot

# Add the legend to the last subplot with adjustments for size and gap
axs[-1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.75), fancybox=True, shadow=True, ncol=2, fontsize=30)

plt.tight_layout()
plt.show()

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from netCDF4 import Dataset

# Station and node configuration remains the same
stations_and_nodes = [
    # {'station': '8632200', 'node': 104851, 'name': 'Kiptopeke, VA'},
    # {'station': '8571892', 'node': 12948, 'name': 'Cambridge, MD'},
    # {'station': '8516945', 'node': 37566, 'name': 'Kings Point, NY'},
    # {'station': '8722956', 'node': 76314, 'name': 'South Port, FL'},
    # {'station': '8720218', 'node': 90753, 'name': 'Mayport, FL'},
    {'station': '8771341', 'node': 21927, 'name': 'GBay, TX'},
    {'station': '8656483', 'node': 88800, 'name': 'Beaufort, NC'},
    # {'station': '8661070', 'node': 103187, 'name': 'Springmaid Pier, SC'},
    # {'station': '8721604', 'node': 89105, 'name': 'Trident Pier, FL'},
    # {'station': '8726724', 'node': 38938, 'name': 'Clearwater Beach, FL'}
]

# Load NetCDF files
withoutoffset_path = '/work2/07174/soelem/validation/base_run/fort.63.nc'
withoffset_path = '/work2/07174/soelem/validation/saral/fort.63.nc'

withoutoffset = Dataset(withoutoffset_path, 'r')
withoffset = Dataset(withoffset_path, 'r')

fig, axs = plt.subplots(len(stations_and_nodes), 1, figsize=(26, 12), dpi=600, sharex=True)

for i, station_info in enumerate(stations_and_nodes):
    node_index = station_info['node'] - 1  # Adjusting node index for Python indexing
    withoutoffset_data = withoutoffset.variables['zeta'][:, node_index]
    withoffset_data = withoffset.variables['zeta'][:, node_index]

    # Find the minimum length to ensure alignment
    min_length = min(len(withoutoffset_data), len(withoffset_data))

    # Adjust the data and dates array lengths
    withoutoffset_data = withoutoffset_data[:min_length]
    withoffset_data = withoffset_data[:min_length]

    # Adjust the phase by shifting
    withoutoffset_data_shifted = np.roll(withoutoffset_data, -4)[:min_length-4]
    withoffset_data_shifted = np.roll(withoffset_data, -4)[:min_length-4]
    dates = pd.date_range(start='2023-07-16', periods=min_length-4, freq='H')

    # NOAA tide data preparation
    url = f'https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?begin_date=20230716&end_date=20231031&station={station_info["station"]}&product=hourly_height&datum=MSL&time_zone=lst&units=metric&format=csv'
    noaa_data = pd.read_csv(url, parse_dates=['Date Time'], usecols=['Date Time', ' Water Level']).dropna()
    noaa_data.set_index('Date Time', inplace=True)
    noaa_data = noaa_data.reindex(dates, method='nearest')

    axs[i].plot(dates, withoutoffset_data_shifted, label='Without Offset', color='blue', linewidth=2)
    axs[i].plot(dates, withoffset_data_shifted, label='With Offset', color='red')
    axs[i].plot(dates, noaa_data[' Water Level'].values[:min_length-4], label='NOAA Observation', color='black', linestyle='--')

    axs[i].set_title(station_info['name'])
    axs[i].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    axs[i].tick_params(axis='x', rotation=45)
    axs[i].grid(True)

# Adjust the last axis to create space for the legend
handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=3)

plt.tight_layout()
plt.subplots_adjust(bottom=0.1)
plt.show()